### In-Context Learning

In [3]:
from openai import OpenAI
import json

In [1]:
api_key = "sk-VUQOgoxNjtiPxFDo895535A3635847B7A903688099089385"
url = "https://api.xi-ai.cn/v1/chat/completions"

In [4]:
file_path = 'error_log.json'
with open(file_path, 'r') as file:
    error_log = json.load(file)
error_log

{'ID': 2270,
 'Question Number': 107,
 'Share Context': '',
 'Share Image': '',
 'Question Text': 'Consider the expected returns and standard deviations for the following portfolios:',
 'Image': 'images/Foundationofriskmanagement1_images/107u.png',
 'Options': {'A': ' Portfolio 1',
  'B': ' Portfolio 2',
  'C': ' Portfolio 3',
  'D': ' Portfolio 4'},
 'Answer': 'A',
 'Explanation': 'Portfolio 1 is not efficient because it has a lower expected return and higher risk than Portfolios 2, 3, and 4. The portfolio is not mean variance efficient due to its suboptimal risk-return profile.',
 'QA Type': 'Knowledge reasoning QA',
 'Question Type': 'text+image',
 'Level of Difficulty': 'Easy',
 'Knowledge Topics': 'mean variance efficiency, portfolio analysis',
 'General Topics': 'Foundation of Risk Management',
 'Book Label': 'foundation of risk management1',
 'Wrong_answer': 'B',
 'Wrong_reasoning_steps': ['Step 1: I focused on finding the portfolio with the highest return for the least risk, wh

In [5]:

file_path = '../dataset/multiDemo.json'
with open(file_path, 'r') as file:
    data = json.load(file)
query=data[84]
query

{'ID': 2537,
 'Question Number': 21,
 'Share Context': '',
 'Share Image': '',
 'Question Text': 'What is the correlation between X & Y?',
 'Image': 'images/QuantitativeAnalysis1_images/21u.png',
 'Options': {'A': ' 0.6330', 'B': ' 0.7044', 'C': ' 0.8175', 'D': ' 0.9286'},
 'Answer': 'D',
 'Explanation': '$$\\text{Covariance}(X, Y) = 0.0520\\%, \\sigma_X = 2.8\\%, \\sigma_Y = 2.0\\%, \\text{Correlation} = \\frac{0.0520\\%}{2.8\\% \\times 2.0\\%} = 0.9286$$',
 'QA Type': 'Math reasoning QA',
 'Question Type': 'text+image',
 'Level of Difficulty': 'Easy',
 'Knowledge Topics': 'correlation, covariance, statistics',
 'General Topics': 'Quantitative Methods',
 'Book Label': 'Quantitative Analysis1'}

#### Unimodal data

In [ ]:
# 给定的示例和反馈信息 (input_data)
input_examples = [
    {
        "question": "Consider the expected returns and standard deviations for the following portfolios...",
        "feedback": "Check the standard deviation formula, and ensure you understand portfolio variance."
    },
    {
        "question": "What is the expected return for portfolio B given the risk-free rate and excess return of portfolio A?",
        "feedback": "Ensure you're using the correct risk-free rate in the calculation."
    }
]

# 定义新的类似题目
new_question = "Given the expected returns and standard deviations of these assets, calculate the optimal portfolio allocation for a risk-averse investor."

# 定义ICL函数
def in_context_learning(examples, new_question):
    # 准备示例输入
    prompt = "Here are examples of samiler solved problems and feedback:\n\n"
    for example in examples:
        prompt += f"Q: {example['question']}\nFeedback: {example['feedback']}\n\n"

    prompt += f"Now solve this new problem based on the examples above:\nQ: {new_question}\n\n"

    # 发送请求给模型
    response = openai.Completion.create(
        engine="gpt-4",  # 使用你选择的模型
        prompt=prompt,
        max_tokens=200,
        temperature=0.7
    )
    
    return response.choices[0].text.strip()

# 执行ICL
output = in_context_learning(input_examples, new_question)
print("Model's Response to New Question:\n", output)



In [ ]:

api_key = "sk-VUQOgoxNjtiPxFDo895535A3635847B7A903688099089385"
url = "https://api.xi-ai.cn/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization":f"Bearer {api_key}"
}
ICL_instruction = f"""
You answered the question incorrectly. Here are examples of similar solved problems and feedback to help you understand the correct approach:
Here is the samilar question:
Share context: {data.get('Share Context')}

Question: {data.get('Question Text')}

Options: {data.get('Options')}
Feedback: {data.get('Feedback')}
"""

task_instruction = f"""
You are a financial expert. You are supposed to answer the given question. First, think through the problem step by step, documenting each necessary step clearly. Then, conclude your response with the final answer in your last sentence as “Therefore, the answer is \\boxed{{}}”.
Here is the new question:
Share context: {data.get('Share Context')}

Question: {data.get('Question Text')}

Options: {data.get('Options')}

Let's think through the problem step by step to answer the given question. The reasoning steps should include math symbols, formulas, and equations properly formatted using Markdown. Ensure that any math symbols or equations are enclosed in inline `\( \)` for smaller formulas or in block `\[ \]` for larger equations. 

### Example output format:
- Use Markdown headers for numbering the steps (e.g., "### step 1. Step Title").
- For inline math symbols or expressions, use `\( \)` (e.g., `\( x = y \)`).
- For block math expressions, use `\[ \]` (e.g., `\[ x = \\frac{{a}}{{b}} \]`).
- The final answer should be placed in `\\boxed{{}}` format, allowing only a single letter for the final answer (e.g., `\\boxed{{A}}`).

Now, answer the question, ensuring that all reasoning steps and equations follow the format above.
Put your final answer in \\boxed{{}} and ensure only one letter (such as "A", "B", or "C") is inside the \\boxed{{}} (e.g., \\boxed{{A}}).
"""

payload = {
  "model":"gpt-4o",
  "messages":[
    {
      "role":"user",
      "content":[
        {
          "type":"text",
          "text": ICL_instruction + task_instruction
        },
        {
          "type":"image_url",
          "image_url":{"url": f"data:image/jpeg;base64,{base64_image}"}
        },
       {
          "type": "text",
          "text": "This image above is associated with the question text."
        }, 

      ]
    }
  ],
  "max_tokens":1000,
}
respon = requests.post(url, headers=headers, json=payload)
respon.json()
